# Problem Statement

In a competitive market like the restaurant industry, understanding the factors that influencer business success is crucial for stakeholder. Utilizing the Yelp dataset, this project aims to investigate the relationship betwenn user engagement (reviews, tips, and check-ins) and business success metrics (review countm ratings) for restaurants.

# Research Objectives
*1. Quantify the correlation between user engagement(reviews, tips, check-ins) and reviews count/average star rating:* This will help us determine if restaurants with higher user engagement experience a corresponding increase in reviews and ratings

*2. Analyze the impact od sentiment on reviews count and average star rating:*  We will investigate if positive sentiment in reviews and tips translates to higher star ratings and potentially influences the total number of reviews left.

*3. Time trends in user engagment:* We will explore if consistent user engagment over time is a stonger indicator of long term success compared to sporadic bursts of activity

# Hypothesis Testing
* Higher levels of user engagement (more reviews, tips, and check-ins) correlate with higher review counts and ratings for restaurants.*

* Positive sentiment expressed in reviews and tips contributes to higher overall ratings and review counts for restaurants.*

* Consistent engagement over time is positively associated with sustained business success for restaurants*

### Importing Libraries

In [1]:
import opendatasets as od
import pandas as pd
import numpy as np
import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import warnings
from datetime import datetime
from geopy.geocoders import Nominatim
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import display
warnings.filterwarnings('ignore')

## DataBase Connection

In [2]:
conn = sqlite3.connect('yelp.db')

In [3]:
tables = pd.read_sql_query("select name from sqlite_master where type = 'table'", con=conn)
tables

,name
0,business
1,checkin
2,review
3,tip
4,user


In [4]:
for table in tables['name']:
    display(pd.read_sql_query(f"select * from {table} limit 5", con = conn))

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food"


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [5]:
display(pd.read_sql_query("select count(1) from business ", con = conn))
display(pd.read_sql_query("select * from business where is_open = 1 and lower(categories) like '%restaurant%'", con = conn))

,count(1)
0,150346


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
2,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"Vietnamese, Food, Restaurants, Food Trucks"
4,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,1,"American (Traditional), Restaurants, Diners, B..."
...,...,...,...,...,...,...,...,...,...,...,...,...
34999,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,ID,83702,43.616590,-116.202383,4.5,998,1,"Bars, Gastropubs, Sandwiches, Nightlife, Resta..."
35000,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,1,"Restaurants, Sandwiches, Convenience Stores, C..."
35001,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R..."
35002,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"Restaurants, Mexican"


In [6]:
business_id = pd.read_sql_query("select business_id, review_count from business where is_open = 1 and lower(categories) like '%restaurant%'", con = conn)

In [7]:
# What is descriptive stats for review count and star rating for business
# avg, min, max, median
Query = f"""select avg(review_count) as avg_review_count, min(review_count) as min_review_count, max(review_count) as max_review_count,
(select review_count from business order by review_count limit 1 offset (select count(1) from business) / 2) as review_median,
avg(stars) as avg_stars, min(stars) as min_stars, max(stars) as max_stars,
(select stars from business order by stars limit 1 offset (select count(1) from business) / 2) as star_median


from business where business_id in {tuple(business_id['business_id'])}"""
display(pd.read_sql_query(Query, con = conn).transpose())

In [8]:
# Lets remove outlier using Inter Quantile Range(IQR)
def remove_outlier(df, col):
    Q1 = df[col].quantile(0.25) 
    Q3 = df[col].quantile(0.75) 
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

In [ ]:
print(len(business_id))
business_id = remove_outlier(business_id, 'review_count')

In [ ]:
Query = f"""select avg(review_count) as avg_review_count, min(review_count) as min_review_count, max(review_count) as max_review_count,
(select review_count from business order by review_count limit 1 offset (select count(1) from business) / 2) as review_median,
avg(stars) as avg_stars, min(stars) as min_stars, max(stars) as max_stars,
(select stars from business order by stars limit 1 offset (select count(1) from business) / 2) as star_median
from business where business_id in {tuple(business_id['business_id'])}"""
display(pd.read_sql_query(Query, con = conn).transpose())

In [ ]:
# Which restaurants has highest number of review?

Query = f"""select name, sum(review_count) as total_review_count, 
            avg(stars) as avg_rating 
            from business 
            where business_id in {tuple(business_id['business_id'])} 
            group by name 
            order by total_review_count desc, avg_rating desc
            limit 10"""
display(pd.read_sql_query(Query, con = conn))

In [ ]:
# Which restaurants has highest number of rating?

Query = f"""select name, sum(review_count) as total_review_count, 
            avg(stars) as avg_rating 
            from business 
            where business_id in {tuple(business_id['business_id'])} 
            group by name 
            order by avg_rating desc
            limit 10"""
display(pd.read_sql_query(Query, con = conn))

##### Review count doesn't provide upper hand to rating of that restaurant

In [23]:
# DO restaurants with higher engagement tend to have higher rating
# Engagement is tip count, review_count and check-in count

In [ ]:
display(pd.read_sql_query(f"""select * from checkin""", con = conn))
checkin_df['count'] = sum([1 for i in checkin[date]])
checkin

In [ ]:
display(pd.read_sql_query(f"""select * from tip order by compliment_count desc""", con = conn))

In [ ]:
Query = f"""select business_id, count(review_count) as total_review_cnt, sum(length(date) - length(replace(date, ',', '')) + 1) as total_checkin, count(text)
            from business b
            inner join  checkin c on b.business_id = c.business_id
            inner join tips t on b.business_id = t.business_id
            where business_id in {tuple(business_id['business_id'])}"""
display(pd.read_sql_query(Query, conn))